In [1]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [2]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_recommenders as tfrs
import pandas as pd
from tensorflow.keras.layers import StringLookup

2024-12-10 09:23:57.976050: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-10 09:23:57.984446: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733797437.994650   13872 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733797437.999065   13872 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-10 09:23:58.013107: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
product = pd.read_csv('product3.csv')

user = pd.read_csv('user3.csv')

In [4]:
# Create TensorFlow datasets from the data
user = tf.data.Dataset.from_tensor_slices({
    "user_id": tf.strings.as_string(user['UserID'].values),  # Ensure user_id is string
    "product_title": user['Title'].values,
    "interaction_type": tf.cast(user['InteractionType'].values, tf.int32)
})

product = tf.data.Dataset.from_tensor_slices({
    "title": product['Title'].values,
    "ingredients": product['Ingredients'].values,
    "combined": product['Combined'].values
})

I0000 00:00:1733797439.320666   13872 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7537 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:
# Fungsi negatif sampling untuk menambahkan data label 0
def add_negative_samples(dataset, product_titles, num_negatives=1):
    def map_fn(x):
        negatives = tf.random.shuffle(product_titles)[:num_negatives]
        negative_samples = tf.data.Dataset.from_tensor_slices({
            "user_id": tf.repeat(x["user_id"], num_negatives),
            "product_title": negatives,
            "interaction_type": tf.zeros([num_negatives], dtype=tf.int32)
        })
        positive_sample = tf.data.Dataset.from_tensors(x)
        return positive_sample.concatenate(negative_samples)

    return dataset.flat_map(map_fn)

In [6]:
# Map function to format the user dataset
user = user.map(lambda x: {
    "user_id": tf.cast(x["user_id"], tf.string),  # Ensure user_id is cast to string
    "product_title": x["product_title"],
    "interaction_type": tf.cast(x["interaction_type"], tf.int32)
})

# Map function to format the product dataset
product = product.map(lambda x: {
    "title": x["title"],
    "ingredients": x["ingredients"],
    "combined": x["combined"]
})

In [7]:

# Shuffle and split the user dataset
total_user_count = len(list(user.as_numpy_iterator()))  # Ensure correct dataset size
train_size = int(0.8 * total_user_count)

shuffled_user = user.shuffle(total_user_count, seed=42, reshuffle_each_iteration=False)

# Split into train and test sets
train_user = shuffled_user.take(train_size)
test_user = shuffled_user.skip(train_size)

# Format train dataset
train_user = train_user.map(lambda x: ({
    "user_id": x["user_id"],
    "product_title": x["product_title"]
}, x["interaction_type"]))

# Format test dataset
test_user = test_user.map(lambda x: ({
    "user_id": x["user_id"],
    "product_title": x["product_title"]
}, x["interaction_type"]))

# Batch dataset untuk pelatihan
train_user = train_user.batch(4096).prefetch(tf.data.AUTOTUNE)
test_user = test_user.batch(4096).prefetch(tf.data.AUTOTUNE)

# Batch the product dataset
batched_product = product.batch(1_000).prefetch(tf.data.AUTOTUNE)

# Extract unique product titles
product_titles = batched_product.map(lambda x: x["title"])
product_titles_list = list(product_titles.as_numpy_iterator())  # Extract product titles as list
unique_product_titles = np.unique(np.concatenate(product_titles_list))  # Get unique titles

# Extract unique user IDs
user_ids = user.map(lambda x: x["user_id"]).batch(1_000_000)
user_ids_list = list(user_ids.as_numpy_iterator())  # Extract user IDs as list
unique_user_ids = np.unique(np.concatenate(user_ids_list))  # Get unique IDs

# Menampilkan contoh produk dan user
print("Unique Product Titles (First 10):", unique_product_titles[:10])
print("Unique User IDs (First 10):", unique_user_ids[:10])

# Membuat pasangan data (user_id, product_title)
user_data = tf.data.Dataset.from_tensor_slices(unique_user_ids)
product_data = tf.data.Dataset.from_tensor_slices(unique_product_titles)

# Gabungkan kedua dataset (user, product) menjadi pasangan
dataset = tf.data.Dataset.zip((user_data, product_data))

# Lakukan batching dataset
batch_size = 4096
batched_dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

2024-12-10 09:24:03.948433: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-12-10 09:24:04.609793: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Unique Product Titles (First 10): [b'     rendang ayam kacang merah' b'    ayam penyet'
 b'    kering balado tempe kacang pedas manis'
 b'    lele goreng krenyes no amis' b'    orek tahu warna warni'
 b'   gulai ikan kakap merah khas padang' b'   pesmol ikan kerapu'
 b'   telur dadar kelapa parut' b'  soto bening daging sapi'
 b'  tahu brokoli pedas manis']
Unique User IDs (First 10): [b'1' b'10' b'11' b'12' b'13' b'14' b'15' b'16' b'17' b'18']


In [8]:
print("Checking dataset pipeline...")
print("Total items in original user dataset:", len(list(user.as_numpy_iterator())))
print("Total items after shuffle:", len(list(shuffled_user.as_numpy_iterator())))
print("Total items in train_user:", len(list(train_user.unbatch().as_numpy_iterator())))
print("Total items in test_user:", len(list(test_user.unbatch().as_numpy_iterator())))

Checking dataset pipeline...


2024-12-10 09:24:06.889628: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Total items in original user dataset: 2000
Total items after shuffle: 2000
Total items in train_user: 1600
Total items in test_user: 400


In [9]:
# Pastikan user_ids dan product_titles berupa string
unique_user_ids = unique_user_ids.astype(str)
unique_product_titles = unique_product_titles.astype(str)

# Buat vocabulary untuk user IDs dan product titles
user_ids_vocabulary = tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None)
product_titles_vocabulary = tf.keras.layers.StringLookup(vocabulary=unique_product_titles, mask_token=None)

In [10]:
# User Model
user_model = tf.keras.Sequential([
    user_ids_vocabulary,  # Map user IDs to integer indices
    tf.keras.layers.Embedding(
        user_ids_vocabulary.vocabulary_size(),
        256,
        embeddings_regularizer=tf.keras.regularizers.l2(1e-5)
    )
])

# Product Model
product_model = tf.keras.Sequential([
    product_titles_vocabulary,  # Map product titles to integer indices
    tf.keras.layers.Embedding(
        product_titles_vocabulary.vocabulary_size(),
        256,
        embeddings_regularizer=tf.keras.regularizers.l2(1e-5)
    )
])

In [11]:
# Functional Model for User and Product
def create_product_retrieval_model(user_model: tf.keras.Model, product_model: tf.keras.Model):
    # Input layers for user and product
    user_input = tf.keras.Input(shape=(), dtype=tf.string, name="user_id")
    product_input = tf.keras.Input(shape=(), dtype=tf.string, name="product_title")

    # Embedding layers for user and product
    user_embeddings = user_model(user_input)
    product_embeddings = product_model(product_input)

    # Normalize embeddings using Lambda layer
    user_embeddings = tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=-1))(user_embeddings)
    product_embeddings = tf.keras.layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=-1))(product_embeddings)

    # Similarity score (dot product between user and product embeddings)
    similarity = tf.keras.layers.Dot(axes=-1)([user_embeddings, product_embeddings])

    # Define the model
    model = tf.keras.Model(inputs=[user_input, product_input], outputs=similarity)
    return model

# Create model
model = create_product_retrieval_model(user_model, product_model)


In [12]:
# Create the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(),  # Assuming binary relevance (1 for relevant, 0 for not relevant)
    metrics=[tf.keras.metrics.BinaryAccuracy()]
             )

early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=500,
    restore_best_weights=True,
    verbose=1
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_id (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 product_title (InputLayer)  [(None,)]                    0         []                            
                                                                                                  
 sequential (Sequential)     (None, 256)                  13056     ['user_id[0][0]']             
                                                                                                  
 sequential_1 (Sequential)   (None, 256)                  442112    ['product_title[0][0]']       
                                                                                              

In [13]:
# Use the callback during training
history = model.fit(
    train_user,
    epochs=1500,
    callbacks=early_stopping,
)

Epoch 1/1500
1/1 [==============================] - 3s 3s/step - loss: 9.3753 - binary_accuracy: 0.0000e+00
Epoch 2/1500
1/1 [==============================] - 0s 33ms/step - loss: 8.9346 - binary_accuracy: 0.0000e+00
Epoch 3/1500


I0000 00:00:1733797458.650414   13958 service.cc:148] XLA service 0x7f53546404e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733797458.650449   13958 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-12-10 09:24:18.653918: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1733797458.662445   13958 cuda_dnn.cc:529] Loaded cuDNN version 90400
I0000 00:00:1733797458.718161   13958 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 [==============================] - 0s 31ms/step - loss: 8.7312 - binary_accuracy: 0.0000e+00
Epoch 4/1500
1/1 [==============================] - 0s 31ms/step - loss: 8.5974 - binary_accuracy: 0.0000e+00
Epoch 5/1500
1/1 [==============================] - 0s 31ms/step - loss: 8.4422 - binary_accuracy: 0.0000e+00
Epoch 6/1500
1/1 [==============================] - 0s 32ms/step - loss: 8.3046 - binary_accuracy: 0.0000e+00
Epoch 7/1500
1/1 [==============================] - 0s 31ms/step - loss: 8.1934 - binary_accuracy: 0.0000e+00
Epoch 8/1500
1/1 [==============================] - 0s 32ms/step - loss: 8.0851 - binary_accuracy: 0.0000e+00
Epoch 9/1500
1/1 [==============================] - 0s 32ms/step - loss: 7.9972 - binary_accuracy: 0.0000e+00
Epoch 10/1500
1/1 [==============================] - 0s 34ms/step - loss: 7.9292 - binary_accuracy: 0.0000e+00
Epoch 11/1500
1/1 [==============================] - 0s 32ms/step - loss: 7.8727 - binary_accuracy: 0.0000e+00
Epoch 12/1500
1/1 [==

In [15]:
def recommend_products(user_id, product_titles, top_k=5):
    """
    Merekomendasikan produk untuk pengguna tertentu berdasarkan similarity score.
    """
    # Dapatkan embedding user
    user_embedding = user_model(tf.constant([user_id]))

    # Dapatkan embedding untuk semua produk
    product_embeddings = product_model(tf.constant(product_titles))

    # Hitung similarity scores menggunakan dot product
    similarity_scores = tf.linalg.matmul(user_embedding, product_embeddings, transpose_b=True)

    # Urutkan skor similarity
    sorted_indices = tf.argsort(similarity_scores, direction="DESCENDING").numpy().flatten()

    # Ambil top_k produk berdasarkan skor tertinggi
    return product_titles[sorted_indices][:top_k]

# **Prediksi Similarity Score**
def predict_similarity(user_id, product_title):
    """
    Menghitung similarity score antara user dan produk tertentu.
    """
    score = model.predict({
        "user_id": tf.constant([user_id]),       # Input user_id
        "product_title": tf.constant([product_title])  # Input product_title
    })
    return score[0]  # Ambil hasil prediksi

# Evaluasi model dengan test dataset
test_loss, test_accuracy = model.evaluate(test_user)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Batch Prediction untuk Data User-Produk
def batch_predict(user_ids, product_titles):
    """
    Melakukan prediksi similarity score untuk batch data user dan produk.
    """
    scores = model.predict({
        "user_id": tf.constant(user_ids),
        "product_title": tf.constant(product_titles)
    })

    for user, product, score in zip(user_ids, product_titles, scores):
        print(f"User: {user}, Product: {product}, Similarity Score: {score}")


1/1 [==============================] - 0s 29ms/step - loss: 6.1321 - binary_accuracy: 0.0300
Test Loss: 6.132138252258301, Test Accuracy: 0.029999999329447746


In [17]:
user_id = "5"  # Ganti dengan UserID aktual
product_titles = unique_product_titles  # Menggunakan daftar produk yang sudah ada
recommendations = recommend_products(user_id, product_titles, top_k=5)
print(f"Rekomendasi produk untuk {user_id}: {recommendations}")

similarity_score = predict_similarity(user_id, product_titles)
print(f"Similarity score antara '{user_id}' dan '{product_titles}': {similarity_score}")

Rekomendasi produk untuk 5: ['bistik rolade sapi' 'nasi goreng cakalang'
 'sambal goreng kentang cornet tahu' 'balado telur' 'lapis daging sapi']
1/1 [==============================] - 0s 15ms/step
Similarity score antara '5' dan '['     rendang ayam kacang merah' '    ayam penyet'
 '    kering balado tempe kacang pedas manis' ... 'udang tumis cabe ijo'
 'woku kepala tongkol' 'yamie ayam']': [-1.0371073e-28  1.4923255e-01  2.6000363e-01 ...  2.1136525e-28
  1.8326138e-29  4.2598671e-01]


In [18]:
model.save("recommender_model.h5")

/home/user-labai-3/venv-master/lib/python3.12/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
# Menyimpan bobot model ke file HDF5
model.save_weights("model_weights.h5")

In [22]:
import json
# Menyimpan arsitektur model ke dalam format JSON
model_json = model.to_json()

# Simpan arsitektur model ke file JSON
with open("model_architecture.json", "w") as json_file:
    json.dump(model_json, json_file)